In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Lectura del CSV de entrenamiento
train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')

# Visualizamos las primeras filas del dataset
train_df.head()

C:\Users\wilson.hernandez\AppData\Local\Temp\ipykernel_3296\4198566822.py:14: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')


,identificador_cliente,cust_age_number,customer_seniority_number,main_city_type,town_id,personal_type,marital_status_type,address_relationship_type,adult_dependants_number,target_type,...,cr_dbt_card_tech_expns_amount,cr_debit_card_tech_op_number,cr_dbt_cd_tourism_expns_amount,cr_debit_cd_tourism_op_number,cr_dbt_cd_clothes_expns_amount,cr_debit_cd_clothes_op_number,cr_dbt_card_prvs_expns_amount,cr_debit_card_prvs_op_number,audit_date,alta_tdc
0,1,68,37,1,11001,1,1.0,3,0.0,1,...,12545000.0,3.0,0.0,0.0,8500000.0,1.0,18852200.0,5.0,2024-11-07T05:18:36.416Z,0
1,2,92,37,1,11001,1,2.0,3,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-07T05:18:36.416Z,0
2,3,70,27,0,25572,1,2.0,2,NaN,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-07T05:18:36.416Z,0
3,4,76,27,0,25290,1,2.0,3,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-07T05:18:36.416Z,0
4,5,90,37,1,5001,1,2.0,5,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-11-07T05:18:36.416Z,0


In [20]:
# Definir variables objetivo e identificación
target = 'alta_tdc'
id_col = 'identificador_cliente'

# Separar características (X) y variable objetivo (y)
X = train_df.drop([target, id_col], axis=1)
y = train_df[target]

# Seleccionar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(exclude=['number']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas (antes de conversión):", cat_cols)

# Convertir columnas categóricas a string
X[cat_cols] = X[cat_cols].astype(str)


Columnas numéricas: ['cust_age_number', 'customer_seniority_number', 'main_city_type', 'marital_status_type', 'address_relationship_type', 'adult_dependants_number', 'target_type', 'customer_asset_amount', 'customer_liability_amount', 'customer_status_type', 'month_withdrawals_number', 'month_payment_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'mth_mbl_app_non_mnty_tr_number', 'month_atm_queries_number', 'atm_made_fin_trans_number', 'atm_made_nfin_tran_number', 'inquiries_digi_channel_number', 'dig_financial_trans_tot_number', 'dig_non_fin_trans_tot_number', 'mth_co_netcash_queries_number', 'mth_co_netc_non_mnty_tr_number', 'mth_co_netcash_mnty_tr_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'mth_pplnetc_non_mnty_tr_number', 'month_h2h_queries_number', 'month_h2h_mnty_tr_number', 'month_h2h_non_mnty_tr_number', 'month_webpage_queries_number', 'mth_webpage_mnty_tr_number', 'm

In [21]:
# Pipeline para variables numéricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Pipeline para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos pipelines con ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# Pipeline completo: preprocesamiento + modelo (inicial sin optimización)
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [ ]:
# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
clf.fit(X_train, y_train)

# Predicción y evaluación
y_pred = clf.predict(X_test)

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))


In [ ]:
!pip install optuna

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install xgboost lightgbm

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   -- ------------------------------------- 6.6/124.9 MB 33.6 MB/s eta 0:00:04
   ------ --------------------------------- 20.4/124.9 MB 49.7 MB/s eta 0:00:03
   ----------- ---------------------------- 36.7/124.9 MB 59.8 MB/s eta 0:00:02
   ---------------- ----------------------- 50.1/124.9 MB 60.2 MB/s eta 0:00:02
   --------------------- ------------------ 67.6/124.9 MB 64.3 MB/s eta 0:00:01
   --------------------------- ------------ 84.7/124.9 MB 66.7 MB/s eta 0:00:01
   ------------------------------- ------- 101.4/124.9 MB 68.2 MB/s eta 0:00:01
   ------------------------------------- - 118.8/124.9 MB 69.6 MB/s eta 0:00:01
   --------------------------------------  124.8/124.9 MB 70.5 MB/s eta 0:00:01
   --------------------------------------- 124.9/124.9 MB 62.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 

In [9]:
#!pip install tensorflow

In [ ]:
import pandas as pd

# Módulos de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score

# Librerías de Deep Learning (Keras / TensorFlow)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1. Lectura del CSV de entrenamiento
train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')

# Definir variables objetivo e identificación
target = 'alta_tdc'
id_col = 'identificador_cliente'

# Separar características (X) y variable objetivo (y)
X = train_df.drop([target, id_col], axis=1)
y = train_df[target]

# Seleccionar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(exclude=['number']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas (antes de conversión):", cat_cols)

# Convertir columnas categóricas a string
X[cat_cols] = X[cat_cols].astype(str)

# 2. Definir pipelines de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos pipelines en un ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# 3. Dividir en conjuntos de entrenamiento y prueba
#    (Lo hacemos antes de ajustar el preprocessor para evitar fuga de datos)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Ajustar el preprocessor con el set de entrenamiento y transformar ambos
preprocessor.fit(X_train_raw)
X_train = preprocessor.transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

# Comprobamos la forma de los datos transformados
print("Shape de X_train después del preprocesamiento:", X_train.shape)
print("Shape de X_test después del preprocesamiento:", X_test.shape)

# 5. Construir un modelo de red neuronal (MLP) con Keras
#    Puedes ajustar libremente número de capas, neuronas, etc.

model = keras.Sequential()

# Capa de entrada (shape = número de features)
model.add(layers.Input(shape=(X_train.shape[1], )))

# Ejemplo de 2 capas ocultas
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))      # Regularización
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.3))

# Capa de salida (1 neurona con sigmoide para binario)
model.add(layers.Dense(1, activation='sigmoid'))

# Compilar la red
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

# 6. Entrenamiento de la red neuronal
#    Aquí podrías usar "class_weight" si el dataset está desbalanceado
#    (ej: class_weight={0:1, 1: 3} si la clase 1 es minoritaria).

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,           # Ajusta según tus necesidades
    batch_size=32,
    verbose=1
)

# 7. Evaluación en el conjunto de prueba
y_pred_probs = model.predict(X_test).ravel()   # Probabilidades
y_pred = (y_pred_probs >= 0.5).astype(int)     # Umbral 0.5

print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
    --------------------------------------- 5.5/390.3 MB 27.9 MB/s eta 0:00:14
   -- ------------------------------------- 20.7/390.3 MB 52.3 MB/s eta 0:00:08
   --- ------------------------------------ 33.0/390.3 MB 55.2 MB/s eta 0:00:07
   ---- ----------------------------------- 46.4/390.3 MB 57.9 MB/s eta 0:00:06
   ----- ---------------------------------- 53.5/390.3 MB 54.0 MB/s eta 0:00:07
   ------ --------------------------------- 60.8/390.3 MB 51.7 MB/s eta 0:00:07
   ------- -------------------------------- 69.2/390.3 MB 48.5 MB/s eta 0:00:07
   -------- ------------------------------- 82.3/390.3 MB 50.0 MB/s eta 0:00:07
   --------- ------------------------------ 95.2/390.3 MB 51.5 MB/s eta 0:00:06
   ----------- --------------------------- 110.1/390.3 MB 53.7 MB/

C:\Users\wilson.hernandez\AppData\Local\Temp\ipykernel_5112\249666656.py:20: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')


Columnas numéricas: ['cust_age_number', 'customer_seniority_number', 'main_city_type', 'marital_status_type', 'address_relationship_type', 'adult_dependants_number', 'target_type', 'customer_asset_amount', 'customer_liability_amount', 'customer_status_type', 'month_withdrawals_number', 'month_payment_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'mth_mbl_app_non_mnty_tr_number', 'month_atm_queries_number', 'atm_made_fin_trans_number', 'atm_made_nfin_tran_number', 'inquiries_digi_channel_number', 'dig_financial_trans_tot_number', 'dig_non_fin_trans_tot_number', 'mth_co_netcash_queries_number', 'mth_co_netc_non_mnty_tr_number', 'mth_co_netcash_mnty_tr_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'mth_pplnetc_non_mnty_tr_number', 'month_h2h_queries_number', 'month_h2h_mnty_tr_number', 'month_h2h_non_mnty_tr_number', 'month_webpage_queries_number', 'mth_webpage_mnty_tr_number', 'm

In [3]:
print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))


Reporte de clasificación en Test:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2178
           1       0.61      0.60      0.60       225

    accuracy                           0.93      2403
   macro avg       0.79      0.78      0.78      2403
weighted avg       0.93      0.93      0.93      2403

Precisión: 0.9267582188930503


In [ ]:
import optuna

def objective(trial):
    # Definir hiperparámetros a optimizar
    n_layers = trial.suggest_int("n_layers", 1, 4)
    n_units = trial.suggest_int("n_units", 32, 256, log=True)
    dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    
    # Construir modelo con estos parámetros
    model = keras.Sequential()
    model.add(layers.Input(shape=(X_train.shape[1],)))

    for _ in range(n_layers):
        model.add(layers.Dense(n_units, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
    
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy', keras.metrics.Recall(name='recall')])  # <- Definir Recall con nombre

    # Entrenamiento
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)
    
    # Ver qué métricas fueron guardadas
    print("Métricas disponibles en history:", history.history.keys())

    # Obtener Recall en validación
    recall = max(history.history.get('val_recall', [0]))  # <- Corregido para evitar KeyError

    return recall

# Ejecutar búsqueda de hiperparámetros
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Mejores hiperparámetros:", study.best_trial.params)


[I 2025-03-04 06:41:43,349] A new study created in memory with name: no-name-b728bc0e-8b0d-4d99-89cf-be9f184f6076
[I 2025-03-04 06:42:02,131] Trial 0 finished with value: 0.5183486342430115 and parameters: {'n_layers': 2, 'n_units': 93, 'dropout_rate': 0.30317525757531044, 'learning_rate': 0.001901963257749413}. Best is trial 0 with value: 0.5183486342430115.


Métricas disponibles en history: dict_keys(['accuracy', 'loss', 'recall', 'val_accuracy', 'val_loss', 'val_recall'])


[I 2025-03-04 06:42:19,763] Trial 1 finished with value: 0.5733944773674011 and parameters: {'n_layers': 3, 'n_units': 38, 'dropout_rate': 0.3381017852543778, 'learning_rate': 0.0011135789628476534}. Best is trial 1 with value: 0.5733944773674011.


Métricas disponibles en history: dict_keys(['accuracy', 'loss', 'recall', 'val_accuracy', 'val_loss', 'val_recall'])


[I 2025-03-04 06:42:35,016] Trial 2 finished with value: 0.3302752375602722 and parameters: {'n_layers': 2, 'n_units': 37, 'dropout_rate': 0.34310948923241913, 'learning_rate': 0.00036838173726339306}. Best is trial 1 with value: 0.5733944773674011.


...

Métricas disponibles en history: dict_keys(['accuracy', 'loss', 'recall', 'val_accuracy', 'val_loss', 'val_recall'])


[I 2025-03-04 06:53:58,008] Trial 28 finished with value: 0.5779816508293152 and parameters: {'n_layers': 4, 'n_units': 78, 'dropout_rate': 0.027136115378165303, 'learning_rate': 0.0027779530753713508}. Best is trial 19 with value: 0.60550457239151.


Métricas disponibles en history: dict_keys(['accuracy', 'loss', 'recall', 'val_accuracy', 'val_loss', 'val_recall'])


[I 2025-03-04 06:54:28,176] Trial 29 finished with value: 0.5596330165863037 and parameters: {'n_layers': 2, 'n_units': 151, 'dropout_rate': 0.21960089398168262, 'learning_rate': 0.0015335362139662029}. Best is trial 19 with value: 0.60550457239151.


Métricas disponibles en history: dict_keys(['accuracy', 'loss', 'recall', 'val_accuracy', 'val_loss', 'val_recall'])
Mejores hiperparámetros: {'n_layers': 4, 'n_units': 126, 'dropout_rate': 0.0008711373345541258, 'learning_rate': 0.006183924373275589}


In [7]:
import pandas as pd

# Módulos de scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, accuracy_score

# Librerías de Deep Learning (Keras / TensorFlow)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 1. Lectura del CSV de entrenamiento
train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')

# Definir variables objetivo e identificación
target = 'alta_tdc'
id_col = 'identificador_cliente'

# Separar características (X) y variable objetivo (y)
X = train_df.drop([target, id_col], axis=1)
y = train_df[target]

# Seleccionar columnas numéricas y categóricas
num_cols = X.select_dtypes(include=['number']).columns.tolist()
cat_cols = X.select_dtypes(exclude=['number']).columns.tolist()

print("Columnas numéricas:", num_cols)
print("Columnas categóricas (antes de conversión):", cat_cols)

# Convertir columnas categóricas a string
X[cat_cols] = X[cat_cols].astype(str)

# 2. Definir pipelines de preprocesamiento
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combinar ambos pipelines en un ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, cat_cols)
])

# 3. Dividir en conjuntos de entrenamiento y prueba
#    (Lo hacemos antes de ajustar el preprocessor para evitar fuga de datos)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4. Ajustar el preprocessor con el set de entrenamiento y transformar ambos
preprocessor.fit(X_train_raw)
X_train = preprocessor.transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

# Comprobamos la forma de los datos transformados
print("Shape de X_train después del preprocesamiento:", X_train.shape)
print("Shape de X_test después del preprocesamiento:", X_test.shape)

# -------------------------------------------------------------------------
# Hiperparámetros recomendados por Optuna
n_layers = 4
n_units = 126
dropout_rate = 0.0008711373345541258
learning_rate = 0.006183924373275589
# -------------------------------------------------------------------------

# 5. Construir el modelo de red neuronal (MLP) con los hiperparámetros encontrados

model = keras.Sequential()
model.add(layers.Input(shape=(X_train.shape[1], )))

# Añadimos n_layers capas ocultas, cada una con n_units neuronas y dropout_rate
for _ in range(n_layers):
    model.add(layers.Dense(n_units, activation='relu'))
    model.add(layers.Dropout(dropout_rate))

# Capa de salida (1 neurona con sigmoide para binario)
model.add(layers.Dense(1, activation='sigmoid'))

# Compilar la red con el learning_rate obtenido
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

# 6. Entrenamiento de la red neuronal
#    Ajusta epochs, batch_size y class_weight según tu dataset
history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=50,           # Puedes ajustarlo
    batch_size=32,
    verbose=1
)

# 7. Evaluación en el conjunto de prueba
y_pred_probs = model.predict(X_test).ravel()   # Probabilidades
y_pred = (y_pred_probs >= 0.5).astype(int)     # Umbral 0.5

print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))


C:\Users\wilson.hernandez\AppData\Local\Temp\ipykernel_5112\1919583287.py:17: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train_df.csv', delimiter=';', thousands='.', decimal=',')


Columnas numéricas: ['cust_age_number', 'customer_seniority_number', 'main_city_type', 'marital_status_type', 'address_relationship_type', 'adult_dependants_number', 'target_type', 'customer_asset_amount', 'customer_liability_amount', 'customer_status_type', 'month_withdrawals_number', 'month_payment_number', 'month_purchase_number', 'm_exit_amount', 'm_entry_amount', 'mth_mbl_app_qry_number', 'mth_mbl_app_mnty_tr_number', 'mth_mbl_app_non_mnty_tr_number', 'month_atm_queries_number', 'atm_made_fin_trans_number', 'atm_made_nfin_tran_number', 'inquiries_digi_channel_number', 'dig_financial_trans_tot_number', 'dig_non_fin_trans_tot_number', 'mth_co_netcash_queries_number', 'mth_co_netc_non_mnty_tr_number', 'mth_co_netcash_mnty_tr_number', 'mth_ppl_netcash_queries_number', 'mth_ppl_netcash_mnty_tr_number', 'mth_pplnetc_non_mnty_tr_number', 'month_h2h_queries_number', 'month_h2h_mnty_tr_number', 'month_h2h_non_mnty_tr_number', 'month_webpage_queries_number', 'mth_webpage_mnty_tr_number', 'm

In [8]:
print("\nReporte de clasificación en Test:")
print(classification_report(y_test, y_pred))
print("Precisión:", accuracy_score(y_test, y_pred))


Reporte de clasificación en Test:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      2178
           1       0.58      0.58      0.58       225

    accuracy                           0.92      2403
   macro avg       0.77      0.77      0.77      2403
weighted avg       0.92      0.92      0.92      2403

Precisión: 0.9209321681231794
